In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['pretrained-bert-including-scripts', 'quora-question-pairs', 'kerasbert']


In [2]:
import numpy as np
import pandas as pd
import os
import sys
import random
import keras
import tensorflow as tf
import json
sys.path.insert(0, '../input/pretrained-bert-including-scripts/master/bert-master')
!cp -r '../input/kerasbert/keras_bert' '/kaggle/working'
BERT_PRETRAINED_DIR = '../input/pretrained-bert-including-scripts/uncased_l-12_h-768_a-12/uncased_L-12_H-768_A-12'
print('***** BERT pretrained directory: {} *****'.format(BERT_PRETRAINED_DIR))
import tokenization  #Actually keras_bert contains tokenization part, here just for convenience

Using TensorFlow backend.


***** BERT pretrained directory: ../input/pretrained-bert-including-scripts/uncased_l-12_h-768_a-12/uncased_L-12_H-768_A-12 *****


In [3]:
## Training data
train_df = pd.read_csv('../input/quora-question-pairs/train.csv')
#train_df = train_df.sample(frac=0.1,random_state = 42)
test_df = pd.read_csv('../input/quora-question-pairs/test.csv')

In [4]:
train_df.is_duplicate.value_counts(normalize=True)

0    0.630802
1    0.369198
Name: is_duplicate, dtype: float64

In [5]:
train_df['question1'] = train_df.question1.astype(str)
train_df['question2']  = train_df.question2.astype(str)
test_df['question1']  = test_df.question1.astype(str)
test_df['question2']  = test_df.question2.astype(str)

train_df['question1_len'] = train_df.question1.apply(lambda x: len(x.split()))
train_df['question2_len']  = train_df.question2.apply(lambda x: len(x.split()))
test_df['question1_len']  = test_df.question1.apply(lambda x: len(x.split()))
test_df['question2_len']  = test_df.question2.apply(lambda x: len(x.split()))

In [6]:
train_df.describe(percentiles = [0.9,0.95,0.99])

,id,qid1,qid2,is_duplicate,question1_len,question2_len
count,404290.000000,404290.000000,404290.000000,404290.000000,404290.000000,404290.000000
mean,202144.500000,217243.942418,220955.655337,0.369198,10.942210,11.181991
std,116708.614502,157751.700002,159903.182629,0.482588,5.428824,6.305246
min,0.000000,1.000000,2.000000,0.000000,1.000000,1.000000
50%,202144.500000,192182.000000,197052.000000,0.000000,10.000000,10.000000
90%,363860.100000,456674.200000,461904.100000,1.000000,18.000000,19.000000
95%,384074.550000,496522.550000,499507.200000,1.000000,22.000000,24.000000
99%,400246.110000,529543.220000,530166.330000,1.000000,29.000000,33.000000
max,404289.000000,537932.000000,537933.000000,1.000000,125.000000,237.000000


In [7]:
lr = 2e-5
weight_decay = 0.001
nb_epochs=1
bsz = 32
maxlen1 = 30
maxlen2 = 30
maxlen = maxlen1+maxlen2+3

In [8]:
train_df.head()

,id,qid1,qid2,question1,question2,is_duplicate,question1_len,question2_len
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0,14,12
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0,8,13
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0,14,10
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0,11,9
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0,13,7


In [9]:
#train_df['comment_text'] = train_df['comment_text'].replace({r'\s+$': '', r'^\s+': ''}, regex=True).replace(r'\n',  ' ', regex=True)
train_lines1, train_lines2, train_labels = train_df['question1'].values, train_df['question2'].values, train_df.is_duplicate.values 
test_lines1, test_lines2 = test_df['question1'].values, test_df['question2'].values
## step parameter 
decay_steps = int(nb_epochs*train_lines1.shape[0]/bsz)
warmup_steps = int(0.0001*decay_steps)

In [10]:
from keras_bert.keras_bert.bert import get_model
from keras_bert.keras_bert.loader import load_trained_model_from_checkpoint
print('begin_build')

config_file = os.path.join(BERT_PRETRAINED_DIR, 'bert_config.json')
checkpoint_file = os.path.join(BERT_PRETRAINED_DIR, 'bert_model.ckpt')
model = load_trained_model_from_checkpoint(config_file, checkpoint_file, training=True,seq_len=maxlen)
#model.summary(line_length=120)

begin_build
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [11]:
from keras import backend as K
class AdamWarmup(keras.optimizers.Optimizer):
    def __init__(self, decay_steps, warmup_steps, min_lr=0.0,
                 lr=0.001, beta_1=0.9, beta_2=0.999,
                 epsilon=None, kernel_weight_decay=0., bias_weight_decay=0.,
                 amsgrad=False, **kwargs):
        super(AdamWarmup, self).__init__(**kwargs)
        with K.name_scope(self.__class__.__name__):
            self.decay_steps = K.variable(decay_steps, name='decay_steps')
            self.warmup_steps = K.variable(warmup_steps, name='warmup_steps')
            self.min_lr = K.variable(min_lr, name='min_lr')
            self.iterations = K.variable(0, dtype='int64', name='iterations')
            self.lr = K.variable(lr, name='lr')
            self.beta_1 = K.variable(beta_1, name='beta_1')
            self.beta_2 = K.variable(beta_2, name='beta_2')
            self.kernel_weight_decay = K.variable(kernel_weight_decay, name='kernel_weight_decay')
            self.bias_weight_decay = K.variable(bias_weight_decay, name='bias_weight_decay')
        if epsilon is None:
            epsilon = K.epsilon()
        self.epsilon = epsilon
        self.initial_kernel_weight_decay = kernel_weight_decay
        self.initial_bias_weight_decay = bias_weight_decay
        self.amsgrad = amsgrad

    def get_updates(self, loss, params):
        grads = self.get_gradients(loss, params)
        self.updates = [K.update_add(self.iterations, 1)]

        t = K.cast(self.iterations, K.floatx()) + 1

        lr = K.switch(
            t <= self.warmup_steps,
            self.lr * (t / self.warmup_steps),
            self.lr * (1.0 - K.minimum(t, self.decay_steps) / self.decay_steps),
        )

        lr_t = lr * (K.sqrt(1. - K.pow(self.beta_2, t)) /
                     (1. - K.pow(self.beta_1, t)))

        ms = [K.zeros(K.int_shape(p), dtype=K.dtype(p)) for p in params]
        vs = [K.zeros(K.int_shape(p), dtype=K.dtype(p)) for p in params]
        if self.amsgrad:
            vhats = [K.zeros(K.int_shape(p), dtype=K.dtype(p)) for p in params]
        else:
            vhats = [K.zeros(1) for _ in params]
        self.weights = [self.iterations] + ms + vs + vhats

        for p, g, m, v, vhat in zip(params, grads, ms, vs, vhats):
            m_t = (self.beta_1 * m) + (1. - self.beta_1) * g
            v_t = (self.beta_2 * v) + (1. - self.beta_2) * K.square(g)
            if self.amsgrad:
                vhat_t = K.maximum(vhat, v_t)
                p_t = m_t / (K.sqrt(vhat_t) + self.epsilon)
                self.updates.append(K.update(vhat, vhat_t))
            else:
                p_t = m_t / (K.sqrt(v_t) + self.epsilon)

            if 'bias' in p.name or 'Norm' in p.name:
                if self.initial_bias_weight_decay > 0.0:
                    p_t += self.bias_weight_decay * p
            else:
                if self.initial_kernel_weight_decay > 0.0:
                    p_t += self.kernel_weight_decay * p
            p_t = p - lr_t * p_t

            self.updates.append(K.update(m, m_t))
            self.updates.append(K.update(v, v_t))
            new_p = p_t

            if getattr(p, 'constraint', None) is not None:
                new_p = p.constraint(new_p)

            self.updates.append(K.update(p, new_p))
        return self.updates

    def get_config(self):
        config = {
            'decay_steps': float(K.get_value(self.decay_steps)),
            'warmup_steps': float(K.get_value(self.warmup_steps)),
            'min_lr': float(K.get_value(self.min_lr)),
            'lr': float(K.get_value(self.lr)),
            'beta_1': float(K.get_value(self.beta_1)),
            'beta_2': float(K.get_value(self.beta_2)),
            'epsilon': self.epsilon,
            'kernel_weight_decay': float(K.get_value(self.kernel_weight_decay)),
            'bias_weight_decay': float(K.get_value(self.bias_weight_decay)),
            'amsgrad': self.amsgrad,
        }
        base_config = super(AdamWarmup, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

In [12]:
from keras.layers import Dense,Input,Flatten,concatenate,Dropout,Lambda
from keras.models import Model
import keras.backend as K
import re
import codecs
adamwarm = AdamWarmup(lr=lr,decay_steps = decay_steps, warmup_steps = warmup_steps,kernel_weight_decay = weight_decay)

sequence_output = model.layers[-6].output
pool_output = Dense(1, activation='sigmoid', kernel_initializer=keras.initializers.TruncatedNormal(stddev=0.02),name = 'real_output')(sequence_output)
model3 = Model(inputs=model.input, outputs=pool_output)
model3.compile(loss='binary_crossentropy', optimizer=adamwarm)
#model3.summary()

In [13]:
def convert_lines(example1, example2, max_seq_length1,max_seq_length2,tokenizer):
    #max_seq_length1=2
    #max_seq_length2-=1
    all_tokens = []
    longer1 = 0
    longer2 = 0
    for i in range(example1.shape[0]):
        tokens_a1 = tokenizer.tokenize(example1[i])
        tokens_a2 = tokenizer.tokenize(example2[i])
        if len(tokens_a1)>max_seq_length1:
            tokens_a1 = tokens_a1[:max_seq_length1]
            longer1 += 1
        if len(tokens_a2)>max_seq_length2:
            tokens_a2 = tokens_a2[:max_seq_length2]
            longer2 += 1
        one_token = tokenizer.convert_tokens_to_ids((["[CLS]"]+tokens_a1))+\
                            [0]*(max_seq_length1-len(tokens_a1))+(tokenizer.convert_tokens_to_ids(["[SEP]"]+tokens_a2)+\
                                              [0]*(max_seq_length2-len(tokens_a2)))+tokenizer.convert_tokens_to_ids(["[SEP]"])
        all_tokens.append(one_token)
    print(longer1)
    return np.array(all_tokens)
    

dict_path = os.path.join(BERT_PRETRAINED_DIR, 'vocab.txt')
tokenizer = tokenization.FullTokenizer(vocab_file=dict_path, do_lower_case=True)
print('build tokenizer done')
print('sample used',train_lines1.shape)
token_input = convert_lines(train_lines1,train_lines2, maxlen1, maxlen2, tokenizer)
seg_input = np.zeros((token_input.shape[0],maxlen))
mask_input = np.ones((token_input.shape[0],maxlen))
print(token_input.shape)
print(seg_input.shape)
print(mask_input.shape)
print('begin training')
class_weight = {0:0.630802, 1:0.369198}
model3.fit([token_input, seg_input, mask_input],train_labels,batch_size=bsz,epochs=nb_epochs, class_weight=class_weight)

build tokenizer done
sample used (404290,)
12602
(404290, 63)
(404290, 63)
(404290, 63)
begin training
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Epoch 1/1
190272/404290 [=============>................] - ETA: 29:59 - loss: 0.1505

In [14]:
token_input_test = convert_lines(test_lines1,test_lines2, maxlen1, maxlen2, tokenizer)
seg_input_test = np.zeros((token_input_test.shape[0],maxlen))
mask_input_test = np.ones((token_input_test.shape[0],maxlen))

preds = model3.predict([token_input_test, seg_input_test, mask_input_test],batch_size=bsz)
submission = pd.DataFrame(index = test_df.index, columns = ['test_id', 'is_duplicate'])
submission['test_id'] = test_df.index
submission['is_duplicate'] = preds
submission.to_csv('submission.csv', index=False)

88100
